In [5]:
# This script installs the necessary Python packages for data analysis and visualization.
%pip install pandas numpy matplotlib seaborn missingno scipy
%pip install openpyxl

import pandas as pd
import glob
import re 
import numpy as np

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
################ Import all necessary data

# Initialize storage
combined_data = []

# Get all Excel files
file_paths = glob.glob("C:/Users/Lenovo/Desktop/Dissertaion/China Data/Sales/*.xlsx")  # Update path

for file in file_paths:
    # Extract year from filename (assuming format like "...2023.xlsx")
    year = re.search(r'\d{4}', file).group()  # Finds first 4-digit number
    
    # Read all sheets from current file
    sheets_dict = pd.read_excel(file, sheet_name=None)
    
    for sheet_name, df in sheets_dict.items():
        # Add identifier columns
        df['year'] = int(year)          # From filename
        df['type'] = sheet_name         # From sheet name
        
        combined_data.append(df)

# Combine all sales data
final_df = pd.concat(combined_data, ignore_index=True)

# Import Price Data
price_file = pd.read_excel(
	"C:/Users/Lenovo/Desktop/Dissertaion/China Data/Prices/Price 2015-202309.xlsx",
	sheet_name="中国汽车分车型每月销售量"
)

# Import Population Data
population_file = pd.read_excel(
    "C:/Users/Lenovo/Desktop/Dissertaion/China Data/Geographical Controls/(10)2000-2023年人口密度.xls",
    sheet_name=None
)

In [13]:
################ Process Price Data

# Define column mapping from Chinese to English
column_mapping = {
    '年份': 'year',
    '月份': 'month',
    '排名': 'rank',
    '车型': 'model',
    '厂商': 'manufacturer',
    '销量': 'sales',
    '售价（万元）': 'price',
    # 'Unnamed: 7': 'extra'  # Optional, if you want to keep this column
}

# Rename columns to English first
price_file = price_file.rename(columns=column_mapping)

# Standardize column names to lowercase with underscores
def standardize_columns(df):
    df.columns = (df.columns
                 .str.lower()  # Convert to lowercase
                 .str.replace(' ', '_')  # Replace spaces with underscores
                 .str.replace('-', '_'))  # Replace hyphens with underscores
    return df

# Apply standardization to both DataFrames
price_file = standardize_columns(price_file)

# Drop irrelevant columns (keep only columns we need)
columns_to_keep_en = ['year', 'month', 'model', 'sales', 'price']
price_file = price_file[columns_to_keep_en].copy()

# Define function to convert price ranges to midpoints
def price_range_to_midpoint(price_str):
    if isinstance(price_str, str):
        # Remove any whitespace
        price_str = price_str.replace(' ', '')
        # Handle range like "9.98-17.98"
        if '-' in price_str:
            parts = price_str.split('-')
            try:
                low = float(parts[0])
                high = float(parts[1])
                return (low + high) / 2
            except ValueError:
                return np.nan
        try:
            return float(price_str)
        except ValueError:
            return np.nan
    return np.nan

# Convert price ranges to midpoints
price_file['price'] = price_file['price'].apply(price_range_to_midpoint)

# Calculate yearly weighted average price
def calculate_weighted_avg(group):
    total_sales = group['sales'].sum()
    weighted_sum = (group['sales'] * group['price']).sum()
    return weighted_sum / total_sales if total_sales != 0 else 0

yearly_weighted_prices = price_file.groupby(['model', 'year']).apply(calculate_weighted_avg).reset_index()
yearly_weighted_prices.columns = ['model', 'year', 'weighted_Avg_Price']

# Add total yearly sales for context
yearly_sales = price_file.groupby(['model', 'year'])['sales'].sum().reset_index()
yearly_weighted_prices = yearly_weighted_prices.merge(yearly_sales, on=['model', 'year'])

# Sort the results
yearly_weighted_prices = yearly_weighted_prices.sort_values(['model', 'year'])

# Display results
print(yearly_weighted_prices)

# Save to Excel
output_path = 'yearly_weighted_prices_by_model.xlsx'
yearly_weighted_prices.to_excel(output_path, index=False)
print(f"Results saved to {output_path}")

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20044\2047349285.py:62: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  yearly_weighted_prices = price_file.groupby(['model', 'year']).apply(calculate_weighted_avg).reset_index()


       model  year  weighted_Avg_Price  sales
0        212  2021                 0.0   1433
1        212  2023                 0.0   1037
2     AION S  2019                 0.0  31929
3     AION S  2020                 0.0  46091
4     AION S  2021                 0.0  69220
...      ...   ...                 ...    ...
5348      魔方  2023                 0.0   5871
5349       鲸  2022                 0.0    463
5350       鲸  2023                 0.0    170
5351     黑金刚  2015                 0.0   1826
5352     黑金刚  2016                 0.0    462

[5353 rows x 4 columns]
Results saved to yearly_weighted_prices_by_model.xlsx


In [ ]:
################ Process Sales Data

# Translate variable names
# Manual translation
translation_map = {
    '省份': 'province',
    '品牌': 'brand',
    '车型': 'model',
    '燃料类型': 'fuel_type',
    '功率': 'power',
    '销量'  : 'sales',
    '总质量': 'mass',
}

# Apply translations
combined_df = final_df.rename(columns=translation_map)

# Standardize column names to lowercase with underscores
combined_df = standardize_columns(combined_df)

# Move 'year' and 'type' to the front
cols = ['year', 'type'] + [col for col in combined_df.columns if col not in ['year', 'type']]
combined_df = combined_df[cols]

# Converts data types
combined_df = combined_df.convert_dtypes()

# View variable names
print(combined_df.columns)


Index(['year', 'type', 'province', 'brand', 'model', 'fuel_type', 'mass',
       'power', 'sales'],
      dtype='object')


In [15]:
################ Combine Sales and Price Data

# Drop sales and month from price_df 
yearly_weighted_prices = yearly_weighted_prices.drop(columns=['sales'], errors='ignore')  # Ignore if 'sales' column doesn't exist

# Merge the datasets on model and year
merged_df = pd.merge(combined_df, yearly_weighted_prices, 
                    on=['model', 'year'], 
                    how='inner')  # Inner join to keep only matching models

# Verify if any models were dropped
original_models = set(combined_df['model'].unique())
merged_models = set(merged_df['model'].unique())
dropped_models = original_models - merged_models

if len(dropped_models) > 0:
    print(f"The following models were dropped due to missing price data: {dropped_models}")
else:
    print("All models had matching price data and were kept.")

# Save the combined data (optional)
merged_df.to_csv('combined_sales_price_data.csv', index=False)

# Display the first few rows of the combined data
print("\nCombined data preview:")
print(merged_df.head())

The following models were dropped due to missing price data: {'御风S16', '斯威G05', '斯威X2', '微蓝 7', '斯威X7', '捷途X70S', '哪吒S EV', '汉 DM-p', '别克Electra E4', '潍柴U75', '5系Li PHEV', '长安CS55 Plus', '比亚迪元Pro', '江淮iEV7S', '力帆乐途', '风行T7', 'EUNIQ 5 EV', '荣威ei6 MAX', '山海L9', '欧尚A500', '起亚EV5', '途胜L', '传祺GM8', '上汽大通EG50', '黑猫', '睿行M60', '现代ix35', '蓝瑟', '风神A60', '上汽大通EUNIQ 7', '江铃E200', '北京U7', '风光E380', 'Velite6', '广汽AIONS MAX', 'FREE', '长安之星2', '科赛3', '朗逸XR', '荣威e550', '卡威E家', '传祺E8', '坦克400', '长安E-Rock', '传祺GM6', 'Velite7', 'S7 EREV', '昌河M70', '康迪EX3', '帝豪EC7', '爱唯欧', '菱智M5 EV', '纳智捷优5 SUV', '新日i03', '众泰M300', '速达SD01', '北京80 TAP', '科赛GT', '奔腾B90', '豪情SUV', '智道U7', '五菱Nano', '嘉年华', '康迪K27', '风光', '比亚迪L3', '北京40L', '奔奔mini', '问界M9', '瑞虎5X', '比亚迪海鸥', '比亚迪E5', '智尚S30', '英致EX1', '北京BW007', '宝骏RC-5W', '408', '帝豪L Hi·P', '哪吒S 增程式', '北京魔方', '成功小虎', '思皓爱跑S', '恒润H23', '奔驰GLC级', '比亚迪海豚', '悦纳RV', '幻速H2', 'MG5', '比亚迪海豹', '帕拉丁', '瑞风M2', '奕泽', '比亚迪汉', '风神AX5', '欧联M1', 'MARVEL X', 'ET5', '嘉悦X8', 'GL8', '众泰Z500', '汉

In [22]:
################ Process Population Data
# Select the correct sheet from the dictionary (replace 'Sheet1' with the actual sheet name if needed)
population_df = population_file['Sheet1'].copy()

# Convert poplation_df to dataframe
population_df = pd.DataFrame(population_df)

In [17]:
################ Cleaning Merged Data

# Converts mass to numeric, handling potential string values or missing data
merged_df['mass'] = pd.to_numeric(merged_df['mass'], errors='coerce').astype(float)

# Function to clean power values
def clean_power(power):
    if isinstance(power, str):
        # Extract all numbers from the string
        numbers = [int(s) for s in power.replace(',', ' ').split() if s.isdigit()]
        if numbers:
            return np.mean(numbers)  # Return average if multiple numbers
        else:
            return np.nan
    return power

# Clean the power column
merged_df['power'] = merged_df['power'].apply(clean_power)


In [18]:
################ Handle missing values and zeros

# Check the initial shape of the merged dataframe
initial_rows = merged_df.shape[0]
print(f"Initial number of rows in merged_df: {initial_rows}")

# Identify missing values in merged_df
missing_values = merged_df.isnull().sum()
print("\nMissing values per column in merged_df:")
print(missing_values)

# Identify zeros in numeric columns of merged_df
# Select numeric columns (excluding year which shouldn't have zeros)
numeric_cols = merged_df.select_dtypes(include=['int64', 'float64']).columns
numeric_cols = [col for col in numeric_cols if col != 'year']

zeros_count = {}
for col in numeric_cols:
    zeros_count[col] = (merged_df[col] == 0).sum()

print("\nZero values in numeric columns of merged_df:")
print(pd.Series(zeros_count))

# Remove rows with missing values or zeros in key columns
# Define which columns to check for zeros (mass, power, sales, weighted_Avg_Price)
cols_to_check = ['mass', 'power', 'sales', 'weighted_Avg_Price']

# Create a mask for rows to keep (no NA and no zeros in specified columns)
mask = (~merged_df[cols_to_check].isnull()).all(axis=1)
for col in cols_to_check:
    mask &= (merged_df[col] != 0)

# Apply the filter to create cleaned_merged_df
cleaned_merged_df = merged_df[mask].copy()

# Report results
removed_rows = initial_rows - cleaned_merged_df.shape[0]
print(f"\nTotal rows removed from merged_df: {removed_rows}")
print(f"New cleaned_merged_df shape: {cleaned_merged_df.shape}")

# Breakdown of why rows were removed
print("\nBreakdown of removed rows:")
print(f"- Rows with missing values: {initial_rows - (~merged_df.isnull()).all(axis=1).sum()}")
for col in cols_to_check:
    print(f"- Rows with zero in {col}: {(merged_df[col] == 0).sum()}")

Initial number of rows in merged_df: 277766

Missing values per column in merged_df:
year                     0
type                     0
province                 0
brand                    0
model                    0
fuel_type                0
mass                    68
power                 4628
sales                    0
weighted_Avg_Price       0
dtype: int64

Zero values in numeric columns of merged_df:
mass                       0
power                      0
sales                      0
weighted_Avg_Price    277766
dtype: int64

Total rows removed from merged_df: 277766
New cleaned_merged_df shape: (0, 10)

Breakdown of removed rows:
- Rows with missing values: 4628
- Rows with zero in mass: 0
- Rows with zero in power: 0
- Rows with zero in sales: 0
- Rows with zero in weighted_Avg_Price: 277766


In [19]:
################ Cleaning Merged Data

# Merge models, compute weighted average power and mass
# Group by all columns except mass, power, sales, and weighted_Avg_Price
group_cols = ['year', 'type', 'province', 'brand', 'model', 'fuel_type']
other_cols = ['mass', 'power', 'sales', 'weighted_Avg_Price']

# Calculate sales-weighted averages for mass and power
def weighted_average(group):
    sales = group['sales']
    total_sales = sales.sum()
    
    # Calculate weighted averages
    weighted_mass = (group['mass'] * sales).sum() / total_sales
    weighted_power = (group['power'] * sales).sum() / total_sales
    
    # Sum sales and take first weighted_Avg_Price (should be same within group)
    return pd.Series({
        'mass': weighted_mass,
        'power': weighted_power,
        'sales': total_sales,
        'weighted_Avg_Price': group['weighted_Avg_Price'].iloc[0] 
    })

# Apply the grouping and weighted average calculation
merged_df = merged_df.groupby(group_cols).apply(weighted_average).reset_index()

# Display results
print(f"\nMerged shape: {merged_df.shape}")
print("\nFirst few merged rows:")
display(merged_df.head())

# Save the cleaned merged data to a CSV file
merged_df.to_csv('cleaned_merged_sales_price_data.csv', index=False)




Merged shape: (79290, 10)

First few merged rows:


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20044\1889728818.py:26: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_df = merged_df.groupby(group_cols).apply(weighted_average).reset_index()


,year,type,province,brand,model,fuel_type,mass,power,sales,weighted_Avg_Price
0,2019,国产新能源乘用车,上海市,东风风行,景逸S50,纯电动,2036.0,90.0,1.0,0.0
1,2019,国产新能源乘用车,上海市,丰田,卡罗拉,插电式汽油混合动力,1975.0,53.0,665.0,0.0
2,2019,国产新能源乘用车,上海市,丰田,雷凌,插电式汽油混合动力,1975.0,53.0,1681.0,0.0
3,2019,国产新能源乘用车,上海市,云度,云度π1,纯电动,1785.0,90.0,1.0,0.0
4,2019,国产新能源乘用车,上海市,云度,云度π3,纯电动,1845.0,90.0,75.0,0.0


In [ ]:
################ Calculating market share using population as total market size
AVERAGE_HOUSEHOLD_SIZE = 2.6 

# Calculate market size (in 10,000 households)
population_df['market_size'] = population_df['年末常住人口(万人)'] / AVERAGE_HOUSEHOLD_SIZE

# Merge market size data
merged_df = merged_df.merge(
    population_df[['省份名称', 'year', 'market_size']],
    how='left',
    left_on=['province', 'year'],
    right_on=['省份名称', 'year']
)

# Clean up after merge - drop the redundant province name column
merged_df = merged_df.drop(columns=['省份名称'])

# Convert to actual household count
merged_df['market_size'] = merged_df['market_size'] * 10000 

# Calculate market share with error handling
merged_df['market_share'] = merged_df['sales'] / merged_df['market_size']




np.float64(13128045.0)